# 1) Preparación previa

### Carga de librerías

In [ ]:
# Se debe instalar ipyleaflet, mpu, openpyxl y requests

import requests
from pandas import json_normalize
import ipyleaflet
import mpu
import pandas as pd
import numpy as np
from ipyleaflet import Map, basemaps, Marker, AwesomeIcon, Icon, FullScreenControl

### Íconos a usar luego

In [ ]:
# Los siguientes íconos serán utilizados para visualizar en el mapa final 

icono_subte = Icon(icon_url='https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Subte-logo.svg/1200px-Subte-logo.svg.png', icon_size=[28, 28])

icon0 = AwesomeIcon(
    name='home',
    marker_color='green',
    icon_color='black',
    spin=False
)

icon1 = AwesomeIcon(
    name='home',
    marker_color='beige',
    icon_color='black',
    spin=False
)

icon2 = AwesomeIcon(
    name='home',
    marker_color='orange',
    icon_color='black',
    spin=False
)

icon3 = AwesomeIcon(
    name='home',
    marker_color='red',
    icon_color='black',
    spin=False
)

### Estaciones a usar luego

In [ ]:
# Dirección web para acceder a los datos del gobierno de la ciudad:
url = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/subte-estaciones/subte_estaciones.geojson'
response = requests.get(url)
dictr = response.json()

# Hacemos referencia a las estaciones dentro del conjunto de datos:
estaciones = dictr['features']
df = json_normalize(estaciones)

# Creamos la columna Longitud:
df['Longitud'] = df['geometry.coordinates'].apply(lambda x:x[0])

# Creamos la columna Latitud:
df['Latitud'] = df['geometry.coordinates'].apply(lambda x:x[1])

# Creamos la máscara para filtrar los de Linea A:
mascara_lineas_elegidas = df['properties.LINEA'].isin(['A', 'B', 'C', 'E', 'H']) 
lineas_elegidas = df.loc[mascara_lineas_elegidas, :]
lineas_elegidas = lineas_elegidas.drop(['type', 'properties.ID', 'properties.LINEA', 'geometry.type', 'geometry.coordinates'], axis = 1)
lineas_elegidas

### Funciones a usar luego

In [ ]:
# La siguiente función nos permite definir la distancia en latitud y longitud respecto de las líneas de subte:
def distancia(lat, long):
    aux = []
    for index, row in lineas_elegidas.iterrows():
        aux.append(
            mpu.haversine_distance(
                (row['Latitud'], row['Longitud']), 
                (lat, long)
            )
        )
    return min(aux)

# La siguiente función nos permite aplicar íconos a los marcadores según su quantil:
def marca_propiedad(lat, long, quantile):
    if quantile == 0:
        icono_prop = icon0
    elif quantile == 1:
        icono_prop = icon1
    elif quantile == 2:
        icono_prop = icon2  
    else: 
        icono_prop = icon3
    marker = Marker(location=(lat,long), draggable=False, icon=icono_prop)
    basic_map.add_layer(marker)

# La siguiente función nos permite limpiar la superficie según las inconsistencias entre la total y la cubierta.
# Se eligií dicha columna con respecto a 'surface_total_in_m2' ya que tenía menor cantidad de registros nulos
def limpieza_superficie(sup_total, sup_cubierta):
    if sup_total is not None and sup_cubierta is not None:
            if sup_total >= sup_cubierta:
                superficie = sup_total
            else: 
                superficie = np.NaN
    elif sup_total is not None:
            superficie = sup_total
    elif sup_cubierta is not None:
            superficie = sup_cubierta
    else: superficie = np.NaN             
                    
    return superficie

### Lectura del dataset original de Properati

In [ ]:
df_properati = pd.read_csv("https://media.githubusercontent.com/media/Agustin-Bulzomi/Projects/main/Programming/Digital%20House/Support%20Files/Project%201/Properati.csv")

# 2) Limpieza del dataset

### Generación de DF solo para Capital

In [ ]:
df_capital = df_properati[df_properati["state_name"] == "Capital Federal"]
df_capital.shape

### Corrección de ortografía de barrios y mapeo de barrios inexistentes

In [ ]:
%%capture output
df_capital["place_name"].replace(['San Cristobal','Barrio Norte', 'Centro / Microcentro'], ['San Cristóbal','Recoleta', 'Monserrat'], inplace=True)

### Lectuura el archivo de Mapping de Barrios por Comuna

In [ ]:
df_barrios_comuna = pd.read_csv("https://raw.githubusercontent.com/Agustin-Bulzomi/Projects/main/Programming/Digital%20House/Support%20Files/Project%201/Barrios_por_comuna.csv")

### Merge del DF de Capital con los barrios según comunas

In [ ]:
df_capital_comuna = pd.merge(df_capital, df_barrios_comuna, how='left', left_on='place_name', right_on='Barrio')
del df_capital_comuna["description"]
del df_capital_comuna["title"]
del df_capital_comuna["properati_url"]
del df_capital_comuna["image_thumbnail"]


In [ ]:
df_capital_comuna["Comuna"].isna().sum()

### Enumeración de los barrios por los que atraviesa la Linea A, elegida como la principal para analizar.

In [ ]:
barrios_linea_a = pd.Series(['Monserrat', 'Balvanera', 'Almagro', 'Caballito'])
barrios_linea_a

### Filtro del DF de Capital según los barrios definidos

In [ ]:
df_capital_barrios_subte = df_capital_comuna[df_capital_comuna['Barrio'].isin(barrios_linea_a)]
df_capital_barrios_subte.shape

### Eliminación de los registros sin coordenadas

In [ ]:
df_latlon_notna = df_capital_barrios_subte[df_capital_barrios_subte['lat-lon'].notna()]

In [ ]:
df_latlon_notna.shape

### Eliminación de los registros sin precios

In [ ]:
df_price_notna = df_latlon_notna[df_latlon_notna['price_usd_per_m2'].notna()]
df_price_notna

### Limpieza de la superficie

In [ ]:
# Se debe corregir inconsistencias entre la superficie total y la cubierta

In [ ]:
%%capture output
df_price_notna["superficie"] = df_price_notna[['surface_total_in_m2','surface_covered_in_m2']].apply(lambda df_price_notna: limpieza_superficie(df_price_notna['surface_total_in_m2'],df_price_notna['surface_covered_in_m2']),axis=1)
df_price_notna.head(4)

# 3) Análisis de hipótesis

### Cálculo de las distancias entre cada propiedad y las 17 estaciones

In [ ]:
%%capture output
df_price_notna['distancia'] = df_price_notna.apply(lambda x: distancia(x['lat'], x['lon']), axis=1)
df_price_notna

### Sampleo aleatorio para obtener una visualización más prolija

In [ ]:
df_randomizado = df_price_notna.sample(n=100, random_state = 1)

### Definición de quantiles

In [ ]:
df_randomizado['quantile'] = pd.qcut(df_randomizado['price_usd_per_m2'], 4, labels=False)
df_randomizado

### Visualización en un mapa

In [ ]:
basic_map = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=(-34.60823238941058, -58.398160858739075), zoom=14)

for index, row in lineas_elegidas.iterrows():
    marker = Marker(location=(row['Latitud'],row['Longitud']), draggable=False, icon=icono_subte)
    basic_map.add_layer(marker);

df_randomizado[['lat','lon','quantile']].apply(lambda x: marca_propiedad(x['lat'],x['lon'],x['quantile']),axis=1)
basic_map.add_control(FullScreenControl())

display(basic_map)

### Exportación del .CSV

In [ ]:
df_randomizado.to_csv("df_randomizado_CABA.csv", index=False)